<a href="https://colab.research.google.com/github/ashioyajotham/Daily-ML/blob/main/AutomaticDownloadSatellite_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective
Satellite Imagery Download : to download batches for the Greek Sea and the Croatia areas.

Install necessary libraries

In [1]:
### install necessary libraries

#! pip install -U scikit-learn
! pip install rasterio
#! pip install holoviews
! pip install hvplot
! pip install contextily
#! pip install matplotlib shapely rich holoviews rioxarray
#! pip install earthengine-api
#! pip install geemap
#! pip install scikit-learn-intelex
#! pip install tifffile
#! pip install scikit-image
#! pip install opencv-python
#! pip install pykml
#! pip install shapely
#! pip install retry
#! pip install pyproj
#! pip install scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.7 MB/s eta 0:00:00


Import necessary libraries

In [2]:
### import necessary libraries

import ee
import geemap
import sklearn
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import fiona
from google.colab import drive
import os
import requests
import shutil
#from retry import retry
import logging
import pyproj

Start google earth engine

In [3]:
your_authentication_gee_project = "data-audio-237110" # add your project name here

ee.Authenticate()
ee.Initialize(project=your_authentication_gee_project)

Connect to google drive and create necessary files

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from google.colab import auth
auth.authenticate_user()

In [6]:
#! mkdir -p "/content/drive/My Drive/Winter_images_2"
#! mkdir -p "/content/drive/My Drive/Water_masks"
#! mkdir -p "/content/drive/My Drive/roi_ee_2"
#! mkdir -p "/content/drive/My Drive/winter_images_2_labels"

Read in csv's with roi's for seagrass areas

In [7]:
### For Croatia:

new = pd.read_csv('/content/drive/MyDrive/Seagrass_Files/roi_Croatia_data_reg_roi.csv')

from shapely.wkt import loads
# Convert the 'geometry' column from text to Shapely geometries
new['geometry'] = new['geometry'].apply(loads)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(new, geometry='geometry')

#gdf = gdf.set_crs('EPSG:3857')

gdf = gdf.set_crs('EPSG:4326')
print(len(gdf))

1330


In [8]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
### subsample Croatia if required

gdf3 = gdf.head(3)

In [10]:
### check size of reprojected polygons:

gdf3['area'] = gdf3.geometry.area
gdf3.area

<ipython-input-10-14e93f55f32b>:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf3['area'] = gdf3.geometry.area
/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
<ipython-input-10-14e93f55f32b>:4: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf3.area


0    6553600.0
1    6553600.0
2    6553600.0
dtype: float64

In [11]:
### check size of reprojected polygons:

area_degrees = 0.000532
# Approximate conversion factor at the equator
conversion_factor = 111320
area_square_meters = area_degrees * (conversion_factor ** 2)

print("Area in square meters:", area_square_meters)

Area in square meters: 6592619.756800001


In [12]:
### subsampling variance two

gdf5 = gdf.iloc[[5,7,8]]
gdf5 = gdf5.reset_index()

In [13]:
### check locations on a map
from shapely.geometry import mapping
# Create a geemap Map instance
M = geemap.Map()

fc1= geemap.geopandas_to_ee(gdf3)

# Add the bounding box geometry to the map
#M.addLayer(AOI4, {'color': 'blue'}, 'Bounding Box Geometry')
M.centerObject(fc1, zoom=10)
M.add_ee_layer(fc1, {'color': 'red'}, 'Polygons from GeoPandas')

# Display the map
M

Map(center=[31.136204034028022, 19.999999999975145], controls=(WidgetControl(options=['position', 'transparent…

In [14]:
### For Greece:

new2 = pd.read_csv('/content/drive/MyDrive/Seagrass_Files/roi_Greece_data_reg_roi.csv')

from shapely.wkt import loads
# Convert the 'geometry' column from text to Shapely geometries
new2['geometry'] = new2['geometry'].apply(loads)

# Convert the DataFrame to a GeoDataFrame
gdf2 = gpd.GeoDataFrame(new2, geometry='geometry')

#gdf2 = gdf2.set_crs('EPSG:3765')

gdf2 = gdf2.set_crs('EPSG:4326')
len(gdf2)

5409

In [15]:
### subsample Croatia if required

gdf4 = gdf2.head(3)
gdf4.geometry

0    POLYGON ((2158509.39712 4840787.62475, 2158509...
1    POLYGON ((2158509.39712 4843347.62475, 2158509...
2    POLYGON ((2158509.39712 4845907.62475, 2158509...
Name: geometry, dtype: geometry

In [16]:
### check area of reprojected polygons

gdf4['area'] = gdf4.geometry.area
gdf4.area

0    6553600.0
1    6553600.0
2    6553600.0
dtype: float64

In [17]:
### check locations on a map
from shapely.geometry import mapping
# Create a geemap Map instance
M2 = geemap.Map()

fc2= geemap.geopandas_to_ee(gdf4)

# Add the bounding box geometry to the map
#M.addLayer(AOI4, {'color': 'blue'}, 'Bounding Box Geometry')
M2.centerObject(fc2, zoom=10)
M2.add_ee_layer(fc2, {'color': 'red'}, 'Polygons from GeoPandas')

# Display the map
M2

Map(center=[-57.34875059490362, -70.60287596251727], controls=(WidgetControl(options=['position', 'transparent…

Define helper functions (check them and adjust output file paths)

In [18]:
from shapely.geometry import mapping
def extract_ee_geometry(geopd, idx):
  line_geometry = geopd.iloc[idx].geometry

  # Convert the geometry to GeoJSON format
  AOI6_2 = mapping(line_geometry)
  AOI6 = ee.Geometry(AOI6_2)
  return AOI6

In [19]:
def call_images_ee(eegeo):
  s2SrCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
  csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
  ROI = eegeo
  QA_BAND = 'cs'
  CLEAR_THRESHOLD = 0.8
  composite = s2SrCollection.filterBounds(ROI).filterDate('2022-12-15', '2023-02-15').linkCollection(csPlus, [QA_BAND]).map(lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)))
 # if not composite.size() > 4:
 #     raise ValueError("Too few images retrieved")
    # Your function logic here
  return composite

In [20]:
def listify(eecoll):
  collection_size = eecoll.size().getInfo()

# Initialize an empty list to store the images
  image_list = []

# Iterate over each index in the image collection
  for i in range(collection_size):
    # Get the image at the current index
    image = ee.Image(eecoll.toList(collection_size).get(i))

    # Append the image to the list
    image_list.append(image)
  if not len(image_list) > 4:
      raise ValueError("Too few images retrieved")
  return image_list

In [21]:

def filter_images_with_valid_bands(image_list, eegeo):
    valid_images = []
    for image in image_list:
        # Compute the mean value for band B2
        band_b2_mean = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=eegeo,  # Assuming AOI6 is properly defined
            scale=30,
            bestEffort=True  # Set bestEffort to True to handle masked pixels
        ).get('B2')

  #      print("Mean value for band B2 in image", image.id().getInfo(), ":", band_b2_mean.getInfo())  # Print band B2 mean for debugging

        # Check if the mean value of band B2 is not None
        if band_b2_mean.getInfo() is None:
#            print("Image", image.id().getInfo(), "has a missing mean value for band B2. Filtering out.")
            pass
        else:
            valid_images.append(image)

    return valid_images

In [22]:
import math

# Function to filter images with valid bands and no NaN values
# Function to filter images based on the percentage of NaN values
# Function to filter images based on the percentage of NaN values
# Function to filter images based on the percentage of NaN values
# Function to filter images based on the percentage of NaN values
def filter_images_with_nans(image_list, eegeo):
    threshold=0.01
    bands_to_check = ['B2', 'B3', 'B4']
    valid_images = []
    for image in image_list:
 #       print("Processing image:", image.id().getInfo())  # Print image ID for debugging

        # Sample pixels for the specified bands in the AOI6 region
        band_pixels = image.select(bands_to_check).sample(region=eegeo, scale=30, numPixels=1000)

        # Function to count NaN values in each band
        def count_nans(band_name):
            band_pixels_filtered = band_pixels.filter(ee.Filter.notNull([band_name]))
            nan_count = band_pixels.size().subtract(band_pixels_filtered.size())
            return nan_count.divide(band_pixels.size())

        # Compute the number of NaN values for each band
        nan_percentages = ee.List(bands_to_check).map(count_nans)
#        print("NaN percentages:", nan_percentages.getInfo())  # Print NaN percentages for debugging

        # Check if any band exceeds the threshold for NaN values
        exceeds_threshold = any([p > threshold for p in nan_percentages.getInfo()])

        # If no band exceeds the threshold, keep the image
        if not exceeds_threshold:
            valid_images.append(image)
        else:
 #           print("Image", image.id().getInfo(), "has NaN values exceeding the threshold. Filtering out.")
            pass

    return valid_images

In [23]:
import random
def sel_satim(im_li):
  if not len(im_li) > 1:
    raise ValueError("Too few images selected")
# Choosing 2 random items from the original list
  random_items = random.sample(im_li, 2)
  return random_items

In [24]:
### ATTENTION: set file paths within function according to your needs -> there are two file paths in here

def saving_im(ra_li, idx, eegeo, country):
  tofile1 = ra_li[0].select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])

  filename1 = "/content/drive/My Drive/Winter_images_2/" + country + "_" + str(idx) + '_1_GeoTIFF.tif'
  image_1 = tofile1.clip(eegeo).unmask()
  geemap.ee_export_image(image_1, filename=filename1, scale=10, crs='EPSG:4326', region=eegeo, file_per_band=False)

  tofile2 = ra_li[1].select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])

  filename2 = "/content/drive/My Drive/Winter_images_2/" + country + "_" + str(idx) + '_2_GeoTIFF.tif'
  image_2 = tofile2.clip(eegeo).unmask()
  geemap.ee_export_image(image_2, filename=filename2, scale=10, crs='EPSG:4326', region=eegeo, file_per_band=False)

In [25]:
def export_im(ra_li, idx, eegeo, country):
  tofile1 = ra_li[0].select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])

  filename1 = "/content/drive/My Drive/Winter_images_1/" + country + "_" + str(idx) + '_1_GeoTIFF.tif'
  image_1 = tofile1.clip(eegeo).unmask()
  image_1 = image_1.reproject(crs='EPSG:4326', scale=10)
  task1 = ee.batch.Export.image.toDrive(
    image=image_1,
    fileNamePrefix=filename1,
    folder='Winter_images_1',
    region=eegeo,
    fileFormat='GeoTIFF',
    scale=10,
    crs='EPSG:4326'
  )
  task1.start()


  tofile2 = ra_li[1].select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])

  filename2 = "/content/drive/My Drive/Winter_images_1/" + country + "_" + str(idx) + '_2_GeoTIFF.tif'
  image_2 = tofile2.clip(eegeo).unmask()
  image_2 = image_2.reproject(crs='EPSG:4326', scale=10)
  task2 = ee.batch.Export.image.toDrive(
    image=image_2,
    fileNamePrefix=filename2,
    folder='Winter_images_1',
    region=eegeo,
    fileFormat='GeoTIFF',
    scale=10,
    crs='EPSG:4326'
  )
  task2.start()


In [26]:
### ATTENTION: set file paths within function according to your needs -> there are two file paths in here

def export_im_through_requests(ra_li, idx, geom, country):
  tofile1 = ra_li[0].select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])
  image_1 = tofile1.clip(geom).unmask()
  image_1 = image_1.reproject(crs='EPSG:4326', scale=10)
  prefix = country + "_" + str(idx) + '_1_GeoTIFF'
#  params = {
#    'count': 100,  # How many image chips to export
#    'buffer': 127,  # The buffer distance (m) around each point
#    'scale': 10,  # The scale to do stratified sampling
#    'seed': 1,  # A randomization seed to use for subsampling.
#    'dimensions': '256x256',  # The dimension of each image chip
#    'format': "png",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
#    'prefix': 'tile_',  # The filename prefix
#    'processes': 25,  # How many processes to used for parallel processing
#    'out_dir': '.',  # The output directory. Default to the current working directly
#}
  url = image_1.getDownloadURL({
                'region': geom,
  #              'dimensions': "256x256",
                'format': "GEO_TIFF",
                'scale': 10,
            })
  ext = "tif"
  r = requests.get(url, stream=True)
  if r.status_code != 200:
    r.raise_for_status()

  out_dir = "/content/drive/My Drive/Winter_images_1/"

  filename = f"{out_dir}/{prefix}.{ext}"
  with open(filename, 'wb') as out_file:
    shutil.copyfileobj(r.raw, out_file)
  print("Done1: ", prefix)

  tofile2 = ra_li[1].select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12'])
  image_2 = tofile2.clip(geom).unmask()
  image_2= image_2.reproject(crs='EPSG:4326', scale=10)
  prefix2 = country + "_" + str(idx) + '_2_GeoTIFF'
#  params = {
#    'count': 100,  # How many image chips to export
#    'buffer': 127,  # The buffer distance (m) around each point
#    'scale': 10,  # The scale to do stratified sampling
#    'seed': 1,  # A randomization seed to use for subsampling.
#    'dimensions': '256x256',  # The dimension of each image chip
#    'format': "png",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
#    'prefix': 'tile_',  # The filename prefix
#    'processes': 25,  # How many processes to used for parallel processing
#    'out_dir': '.',  # The output directory. Default to the current working directly
#}
  url2 = image_2.getDownloadURL({
                'region': geom,
  #              'dimensions': "256x256",
                'format': "GEO_TIFF",
                'scale': 10,
            })
  ext2 = "tif"
  r2 = requests.get(url2, stream=True)
  if r2.status_code != 200:
    r2.raise_for_status()

  out_dir2 = "/content/drive/My Drive/Winter_images_1/"

  filename2 = f"{out_dir2}/{prefix2}.{ext2}"
  with open(filename2, 'wb') as out_file2:
    shutil.copyfileobj(r2.raw, out_file2)
  print("Done2: ", prefix2)

In [27]:
### ATTENTION: set file paths within function according to your needs

def save_ROI_as_geojson(eegeo, country, idx):
# Convert the Earth Engine geometry to a feature
  feature_ROI = ee.Feature(eegeo, {})

# Create a feature collection
  fc_ROI = ee.FeatureCollection([feature_ROI])

# Specify the path to save the shapefile
  output_shp1 = "/content/drive/My Drive/roi_ee_2/" + country + "_" + str(idx) + "_1_gj.geojson"

# Export the feature collection as a shapefile
  geemap.ee_export_vector(fc_ROI, filename=output_shp1)

# Specify the path to save the shapefile
  output_shp2 = "/content/drive/My Drive/roi_ee_2/" + country + "_" + str(idx) + "_2_gj.geojson"

# Export the feature collection as a shapefile
  geemap.ee_export_vector(fc_ROI, filename=output_shp2)

In [28]:
def store_watermask(image, eegeo, country, idx):
  # Load an image
  roi = eegeo

# Load an image
  image2 = image

# Create your mask based on band ratios
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')  # NDWI (Normalized Difference Water Index) calculation
  water_threshold = 0  # Threshold value to distinguish water; adjust as needed -> some say needs to be 0 or 0.3
  water_mask = ndwi.gte(water_threshold)

# For example, converting a numpy array mask to an EE Image
  mask_ee = ee.Image(water_mask)

# Convert the mask to vector (polygon) features
  features = mask_ee.reduceToVectors(scale=30, geometry=eegeo)

# Convert the vector features to a FeatureCollection
  feature_collection = ee.FeatureCollection(features)
  # Specify the path to save the shapefile
  output_shp1 =  "/content/drive/My Drive/Water_masks/" + country + "_" + str(idx) + "_1_watermask_geojson.geojson"

# Export the feature collection as a shapefile
  geemap.ee_export_vector(feature_collection, filename=output_shp1)

  output_shp2 =  "/content/drive/My Drive/Water_masks/" + country + "_" + str(idx) + "_2_watermask_geojson.geojson"

# Export the feature collection as a shapefile
  geemap.ee_export_vector(feature_collection, filename=output_shp2)

Do the actual downloads (have you checked output file paths in helper functions? Have you chosen an appropriate gdf? Have you changed the country name string variable below?)

In [29]:
### first create the actual output folders needed:

#! mkdir -p "/content/drive/My Drive/Winter_images_2"
#! mkdir -p "/content/drive/My Drive/Water_masks"
#! mkdir -p "/content/drive/My Drive/roi_ee_2"
#! mkdir -p "/content/drive/My Drive/winter_images_2_labels"

### loop through the gdf of your choice to download satellite images

for idx, row in gdf4.iterrows(): # change name of gdf file if required
  geom = extract_ee_geometry(gdf4, idx) # change name of gdf file if required
  comp = call_images_ee(geom)
  try:
    comp2 = listify(comp)
  except: continue
  valid_comp1 = filter_images_with_valid_bands(comp2, geom)
  if not len(valid_comp1) > 2: continue
  valid_comp2 = filter_images_with_nans(valid_comp1, geom)
  try:
    final_comp = sel_satim(valid_comp2)
  except: continue
  saving_im(final_comp, idx, geom, "CroatiaN") # attention -> change country according to need
  # use function below if saving_im is not working -> set out files path directories within functions correctly
#  export_im_through_requests(final_comp, idx, geom, "CroatiaN") # attention -> change country according to need
  # set out files path directories within functions correctly
  save_ROI_as_geojson(geom, "CroatiaN", idx) # attention -> change country according to need

EEException: User memory limit exceeded.

Create masks/labels

In [ ]:
### import necessary libraries

import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import shape
import geopandas as gpd
from rasterio.mask import mask
from shapely.geometry import mapping
from rasterio.features import geometry_mask
from rasterio.plot import show
import holoviews as hv
import hvplot.xarray
import xarray as xr
import hvplot.pandas
from shapely.geometry import box
from shapely.ops import unary_union
import os
from rasterio.enums import Resampling
import contextily
from rasterio.mask import mask
import subprocess
import rasterio as rio
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona
from rasterio.transform import from_bounds

In [ ]:
### read in seagrass and land gdf - depending on whether you use the Greek or the Croatian dataset
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Reading seagrass polygons into Geopandas dataframe
seagrass_gdf = gpd.read_file('/content/drive/My Drive/Seagrass_Files/Croatia_filtereddataset_4326(1).geojson')
# for Croatia: gpd.read_file('/content/drive/My Drive/Seagrass_Files/Croatia_filtereddataset_4326(1).geojson')
# for Greece: gpd.read_file('/content/drive/My Drive/Seagrass_Files/Greek_Seagrass_Meadows_v0906/Greek Seagrass meadows v0906.shp')
seagrass_gdf = seagrass_gdf[seagrass_gdf.is_valid]
# Read in landmask
shape_path1 = '/content/drive/My Drive/Seagrass_Files/landmask2_Greece.geojson'
# for Croatia: landmask2_Croatia.geojson
gdf_land_mask = gpd.read_file(shape_path1)

In [ ]:
### check if geotif folder and region of interest folder contain the same amount of files
import os

folder_path = "/content/drive/My Drive/Winter_images_2"
folder_path2 = "/content/drive/My Drive/roi_ee_2"

# List files in the first directory
files_in_folder1 = os.listdir(folder_path)

# List files in the second directory
files_in_folder2 = os.listdir(folder_path2)

# Compare the lengths of the lists
if len(files_in_folder1) == len(files_in_folder2):
    print("Both folders contain the same amount of files.")
else:
    print("The folders contain different amounts of files.")


Define helper functions:

In [ ]:
def read_in_aoi(shape_path):
  gdf_sh = gpd.read_file(shape_path)

  # Extract the polygon from the GeoDataFrame
  aoi_polygon = gdf_sh['geometry'].iloc[0]
  return aoi_polygon

In [ ]:
def seagrass_and_land_polygons(seagrass_gdf, gdf_land_mask, shape_path):
  gdf_sh = gpd.read_file(shape_path)
  seagrass_in_roi_1 = gpd.sjoin(seagrass_gdf, gdf_sh, how='inner', op='intersects')
  land_in_roi_1 = gpd.sjoin(gdf_land_mask, gdf_sh, how='inner', op='intersects')
  return seagrass_in_roi_1, land_in_roi_1

Do the actual mask generation

In [ ]:
import os

folder_path = "/content/drive/My Drive/Winter_images_2"
folder_path2 = "/content/drive/My Drive/roi_ee_2"
mask_output_path = "/content/drive/My Drive/winter_images_2_labels"

# List files in the first directory
files_in_folder1 = sorted(os.listdir(folder_path))  # Sorting for consistent iteration

# List files in the second directory
files_in_folder2 = sorted(os.listdir(folder_path2))  # Sorting for consistent iteration

# Iterate over the files in both directories simultaneously
for img_file, shape_file in zip(files_in_folder1, files_in_folder2):
  # Construct full file paths
  geotif_file_path = os.path.join(folder_path, img_file)
  aoi_shape_path = os.path.join(folder_path2, shape_file)
  aoi_polygon = read_in_aoi(aoi_shape_path)
  seagrass_in_roi_1, land_in_roi_1 = seagrass_and_land_polygons(seagrass_gdf, gdf_land_mask, aoi_shape_path)

  with rio.open(geotif_file_path) as src:
    # Read the geospatial metadata
    geotransform = src.transform
    projection = src.crs
    # Read all bands as a numpy array
    bands_data = []
    for i in range(1, src.count + 1):
      band_data = src.read(i)
      bands_data.append(band_data)

    # Stack bands into one numpy array
    stacked_data = np.stack(bands_data, axis=-1)
    # Generate variables later needed for creating masks
    _, cropped_transform1 = mask(src, [mapping(aoi_polygon)], crop=False)
    blue_height1, blue_width1 = src.shape


  seagrass_in_roi_1 = seagrass_in_roi_1.to_crs(projection)
  final_mask_Posidonia = geometry_mask(seagrass_in_roi_1.geometry, out_shape=(blue_height1, blue_width1), transform=cropped_transform1, invert=True)
  land_in_roi_1 = land_in_roi_1.to_crs(projection) ## add needed final gdf here
  if len(land_in_roi_1)> 0:
    final_mask_Land = geometry_mask(land_in_roi_1.geometry, out_shape=(blue_height1, blue_width1), transform=cropped_transform1, invert=True)
  else: final_mask_Land = np.zeros(final_mask_Posidonia.shape, dtype=bool)
  final_mask_water = ~(final_mask_Posidonia | final_mask_Land)
  # Determine the shape of mask_class0
  mask_shape = final_mask_Posidonia.shape

  # Create an empty array to store the combined result with the same shape as mask_class0
  combined_mask = np.zeros(mask_shape, dtype=np.uint8)

  # Assign class labels to the combined mask
  combined_mask[final_mask_Posidonia] = 0  # Class 0
  combined_mask[final_mask_water] = 1  # Class 1
  combined_mask[final_mask_Land] = 2  # Class 2

  # Define the output file path
  output_tif_path = mask_output_path + "/"+ img_file[:-4] + '_mask.tif'

  # Define the transformation matrix
  transform = cropped_transform1

  # Define the CRS (coordinate reference system) of the transformation
  crs = "EPSG:4326"  # Assuming WGS84 coordinate reference system

  # Write the combined mask to a GeoTIFF file
  with rio.open(
    output_tif_path,
    'w',
    driver='GTiff',
    height=combined_mask.shape[0],
    width=combined_mask.shape[1],
    count=1,
    dtype=combined_mask.dtype,
    crs=crs,
    transform=transform
    ) as dst:
      dst.write(combined_mask, 1)


Check some masks or images (adjust folders for it...)

In [ ]:
# Import os module
import os

# Define the path to the folder
folder_path = "/content/drive/My Drive/Winter_images_2"

# List the contents of the folder
folder_contents = os.listdir(folder_path)

# Display the contents
print("Contents of the folder:")
for item in folder_contents:
    print(item)

In [ ]:
### check a single image:
from skimage import io
from skimage.segmentation import felzenszwalb, slic, quickshift
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
import cv2

image = "/content/drive/My Drive/Winter_images_2/Croatia_0_1_GeoTIFF.tif"
img = io.imread(image)
img.shape

In [ ]:
### display image

import numpy as np
import matplotlib.pyplot as plt
from skimage import io

io.imshow(img[:,:,1], cmap='viridis')
plt.show()

Clip images and masks to actual size needed

In [ ]:
### make final files where to store images and labels in 256*256 pixels format

! mkdir -p "/content/drive/My Drive/clipped_Winter_images_2"
! mkdir -p "/content/drive/My Drive/clipped_winter_images_2_labels"

In [ ]:
### check if geotif folder and labels folder contain the same amount of files
import os

folder_path = "/content/drive/My Drive/Winter_images_2"
folder_path2 = "/content/drive/My Drive/winter_images_2_labels"

# List files in the first directory
files_in_folder1 = os.listdir(folder_path)

# List files in the second directory
files_in_folder2 = os.listdir(folder_path2)

# Compare the lengths of the lists
if len(files_in_folder1) == len(files_in_folder2):
    print("Both folders contain the same amount of files.")
else:
    print("The folders contain different amounts of files.")

In [ ]:
### define helper function

import rasterio
from rasterio.windows import Window
import numpy as np

def clip_geotif(rifile, output_path, country, artype, idx):
# Open the GeoTiff image
  with rasterio.open(rifile) as src:
    # Read the entire image as a numpy array
    image_array = src.read()

    # Define the window to extract the upper-left subset (adjust as needed)
    window = Window(0, 0, 256, 256)

    # Read the subset from the original image
    upper_left_subset = src.read(window=window)

    # Get metadata from the original GeoTiff
    meta = src.meta

    # Update metadata with new dimensions and affine transformation
    meta['width'] = 256
    meta['height'] = 256
    meta['transform'] = rasterio.windows.transform(window, src.transform)

    output_file = output_path + "/" + country + "_" + artype + "_" + idx + ".tif"
    # Write the subset as a new GeoTiff
    with rasterio.open(output_file, 'w', **meta) as dst:
        dst.write(upper_left_subset)

#print("Subset saved successfully.")


In [ ]:
### convert images and labels to 256*256 pixels format
import os

folder_path = "/content/drive/My Drive/Winter_images_2"
folder_path2 = "/content/drive/My Drive/winter_images_2_labels"
image_output_path = "/content/drive/My Drive/clipped_Winter_images_2"
label_output_path = "/content/drive/My Drive/clipped_winter_images_2_labels"

country = "Greece" # change if needed

# List files in the first directory
files_in_folder1 = sorted(os.listdir(folder_path))  # Sorting for consistent iteration

# List files in the second directory
files_in_folder2 = sorted(os.listdir(folder_path2))  # Sorting for consistent iteration

# Iterate over the files in both directories simultaneously
for idx, (img_file, label_file) in enumerate(zip(files_in_folder1, files_in_folder2)):
    # idx is the index of the current iteration
  artype = "image"
  ### check the clip_geotif lines -> it may be (clip_geotif(os.path.join(folder_path,img_file), image_output_path, country, artype, str(idx))----)
  clip_geotif(os.path.join(folder_path,img_file), image_output_path, country, artype, str(idx))
  artype = "label"
  clip_geotif(os.path.join(folder_path2,label_file), label_output_path, country, artype, str(idx))

Uploading images and masks to dagshub

In [ ]:
!pip install dagshub -qq

In [ ]:
!dagshub login

In [ ]:
from dagshub.upload import Repo

# Defining the Repo & directory
repo = Repo("Omdena", "TriesteItalyChapter_MappingSeagrassMeadows")
ds = repo.directory('./data')

image_output_path = "/content/drive/My Drive/clipped_Winter_images_2"
label_output_path = "/content/drive/My Drive/clipped_winter_images_2_labels"

# Addes images & masks to data folder
img_list = sorted(os.listdir(image_output_path))
mask_list = sorted(os.listdir(label_output_path))

for i in img_list:
  ds.add(file=os.path.join(image_output_path, i), path=f'./Winter_data/images/{i}')
for i in mask_list:
  ds.add(file=os.path.join(label_output_path, i), path=f'./Winter_data/masks/{i}')

# Commit the changes
ds.commit('Adds images & masks to data folder', versioning='dvc')